In [1]:
import pandas as pd
from datasets import load_dataset

from src.metrics import QAMetricModel, compute_f1_precision_recall, get_tokens, normalize_answer

/fs01/home/snajafi/codes/llm-research/llm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Mon Jun 17 23:27:36 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:86:00.0 Off |                    0 |
|  0%   27C    P8    29W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
hg_token = ""

In [3]:
!huggingface-cli login --token hg_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /h/snajafi/.cache/huggingface/token
Login successful


In [4]:
df = pd.read_csv("/scratch/ssd004/scratch/snajafi/checkpoints/llama3-squadv2.0_normal_icl/llama3.squad2.dev.results.csv")
data = df.to_dict(orient="records")

dataset = load_dataset("rajpurkar/squad_v2", split="validation")
gold_answers = []
for row in dataset:
    if row["answers"]["text"]:
        gold_answers.append(row["answers"]["text"])
    else:
        gold_answers.append(["<no_answer>"])

predictions = []
for row in data:
    predictions.append(row["potential_answer"])

print(len(predictions))
print(len(gold_answers))

print(predictions[0])
print(gold_answers[0])

Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 364517.86 examples/s]


11873
11873
assistant

France
['France', 'France', 'France', 'France']


In [7]:
sentence_t5_metric_model = QAMetricModel(device="cuda:0", batch_size=16, metric_type="sentence_t5")

In [6]:
answers = [[normalize_answer(an) for an in ans] for ans in gold_answers]

In [7]:
preds = [normalize_answer(pred) for pred in predictions]
print(predictions[0:10])
print(preds[0:10])
print(answers[:10])
print(gold_answers[:10])

['assistant\n\nFrance', 'assistant\n\nThe correct answer is "in the 10th and 11th centuries".', 'assistant\n\nBased on the passage, the correct answer is:\n\nDenmark, Iceland, and Norway', 'assistant\n\nRollo', 'assistant\n\nThe shortest continuous text span from the passage that serves as an answer to the question is:\n\n"...the distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century..."\n\nTherefore, the correct answer is:\n\n10th century', 'assistant\n\nThe shortest continuous text span that serves as an answer to the given question is:\n\n"The Normans"\n\nThis answer is justified by the passage, which states that the Normans were the people who gave their name to Normandy in the 10th and 11th centuries.', 'assistant\n\nBased on the passage, the correct answer is:\n\nFinal Answer_11: Normandy', 'assistant\n\nThe correct answer is Rollo.', 'assistant\n\nThe passage does not mention the Frankish identity emerging at a specific time.

In [14]:
scores = sentence_t5_metric_model.compute_metric(preds[:10], answers[:10])

In [16]:
print(scores)

tensor([1.0000, 0.9862, 1.0000, 1.0000, 0.7817, 0.5782, 0.6214, 0.6651, 1.0000,
        1.0000], device='cuda:0')


In [17]:
get_tokens("This question is not answerable.")

['this', 'question', 'is', 'not', 'answerable']

In [18]:
compute_f1_precision_recall(preds[0], answers[0][0])

[1.0, 1.0, 1.0]

In [19]:
print(compute_f1_precision_recall(preds[9], answers[9][0]))
print(preds[9])
print(answers[9][0])

[1.0, 1.0, 1.0]
william conqueror
william conqueror


In [20]:
print(compute_f1_precision_recall(preds[8], answers[8][0]))
print(preds[8])
print(answers[8][0])

[1, 1, 1]
This question is not answerable
This question is not answerable


In [8]:
# load the new llm2vec sentence similarity metrics.
llm2vec_metric_model = QAMetricModel(device="cuda:0", batch_size=16, metric_type="llm2vec")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp:
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp:
- modeling_llama_encoder.py
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|██████████| 4/4 [00:34<00:00,  8.62s/it]
Some weights of the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct were not used when initializing LlamaEncoderModel: ['lm_head.weight']
- This IS expected if you are ini

In [9]:
scores = llm2vec_metric_model.compute_metric(preds[:10], answers[:10])

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.99it/s]


In [10]:
print(scores)

tensor([0.9999, 0.9904, 0.9999, 0.9998, 0.5406, 0.3153, 0.2691, 0.1912, 0.9999,
        0.9999])
